# Absolute Stability Examples

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.sparse as sp
import scipy.sparse.linalg as sla
import fd_tools as fd

## Introduction

When boundary conditions are involved, von Neumann analysis doesn't tell the whole story.  For standard single-step and multi-step methods, we can try to ensure that the values $\Delta t\lambda$ lie inside the absolute stability region of the method.  Often, a good first step is to numerically compute the eigenvalues.  Below is a function that builds a finite difference approximation using $2w+1$ values at each point.  In the middle of the domain, the centered difference is used, near the boundaries, the stencils are biased, but still centered as much as possible.

For example, if $w=1$ and the $\{x_j\}$ are arranged in an equispaced grid, then the derivative approximation produced will be

$$
D = \frac{1}{\Delta x}
\begin{bmatrix}
-\frac{3}{2} & 2 & -\frac{1}{2} \\
-\frac{1}{2} & 0 & \phantom{-}\frac{1}{2} \\
& -\frac{1}{2} & 0 & \phantom{-}\frac{1}{2} \\
& & -\frac{1}{2} & 0 & \phantom{-}\frac{1}{2} \\
& & & \ddots & \ddots & \ddots \\
& & & & -\frac{1}{2} & \phantom{-}0 & \phantom{-}\frac{1}{2} \\
& & & & \phantom{-}\frac{1}{2} & -2 & \phantom{-}\frac{3}{2} \\
\end{bmatrix}
$$

In [ ]:
def first_deriv_matrix(x, w):
    
    I, J, data = [], [], []
    n = len(x)
    
    # bias to the right until we can center
    p = (2*w+1)
    for i in range(w):
        I.extend(p*[i])
        J.extend(range(p))
        data.extend(fd.fd_coeff(1, x[i], x[:p]))
        
    # now fill in centered differences
    for i in range(w, n-w-1):
        I.extend(p*[i])
        J.extend(range(i-w, i+w+1))
        data.extend(fd.fd_coeff(1, x[i], x[i-w:i+w+1]))

    # bias to the left when we can't center anymore
    for i in range(n-w-1, n):
        I.extend(p*[i])
        J.extend(range(n-p, n))
        data.extend(fd.fd_coeff(1, x[i], x[n-p:]))
        
    return sp.csr_matrix((data, (I, J)))

In [ ]:
def matrix_demo():
    x = np.linspace(0, 9, 10)
    D = first_deriv_matrix(x, 2)
    print(np.round(D.toarray(), 3))
    plt.spy(D)
    
matrix_demo()

## Eigenvalues of what?

In the MOL framework, we now want to solve the problem
$$
U' = -aDU,\qquad U(0) = \eta,\qquad U_0(t) = g(t).
$$
As we discussed when talking about absolute stability, each of the standard ODE methods we've covered has an absolute stability region $\mathcal{S}$ in the complex plane.  For stability of the PDE solver, we need to choose the time step $\Delta t$ so that $\Delta t\lambda\in\mathcal{S}$ for every eigenvalue $\lambda$ of our spatial operator.  But what exactly do we need to find the eigenvalues of?

You might think that we need to look at the eigenvalues of $-aD$, but that's not quite right, since it doesn't take into account the boundary conditions on the left.  There are several ways of seeing what we really want to find the eigenvalues of.  Here's one way.  Suppose we are solving the advection problem using forward Euler in time and using the second-order accurate derivative matrix produced by the function above (with $w=1$).  Here is some pseudocode implementing that idea.

```python
# initialize u
u = eta
for it in range(1,num_iter+1):
    
    # update u using forward Euler
    u = u - a*dt*D*u
    
    # fix the boundary conditions
    u[0] = g(it*dt)
```

If we write the iteration scheme out carefully, it looks something like this:
\begin{align*}
U_0^{j+1} &= g(t_{j+1})\\
U_1^{j+1} &= U_1^{j} - \frac{a\Delta t}{2\Delta x}\left(U_2^j - g(t_j)\right)\\
U_2^{j+1} &= U_2^{j} - \frac{a\Delta t}{2\Delta x}\left(U_3^j - U_1^j\right)\\
U_3^{j+1} &= U_3^{j} - \frac{a\Delta t}{2\Delta x}\left(U_4^j - U_2^j\right)\\
&\vdots\\
U_{n-1}^{j+1} &= U_{n-1}^{j} - \frac{a\Delta t}{2\Delta x}\left(U_n^j - U_{n-2}^j\right)\\
U_{n}^{j+1} &= U_{n}^{j} - \frac{a\Delta t}{2\Delta x}\left(3U_n^j -4U_{n-1}^j + U_{n-2}^j\right)\\
\end{align*}

If we think of the vector
$$
\tilde{U}^j = 
\begin{bmatrix}
U_1^j \\ U_2^j \\ \vdots \\ U_n^j
\end{bmatrix}
$$
then, on each step $\tilde{U}$ is updated by the formula
$$
\tilde{U}^{j+1} = \tilde{U}^j - a\Delta t\tilde{D}\tilde{U}^j + b^j,
$$
where
$$
b^j = \begin{bmatrix}\frac{a\Delta t}{2\Delta x}g(t_j) \\ 0 \\ 0 \\ \vdots \\ 0\end{bmatrix}
$$
and
$$
\tilde{D} = 
\frac{1}{\Delta x}
\begin{bmatrix}
0 & \phantom{-}\frac{1}{2} \\
 -\frac{1}{2} & 0 & \phantom{-}\frac{1}{2} \\
 & -\frac{1}{2} & 0 & \phantom{-}\frac{1}{2} \\
 & & \ddots & \ddots & \ddots \\
 & & & -\frac{1}{2} & \phantom{-}0 & \phantom{-}\frac{1}{2} \\
 & & & \phantom{-}\frac{1}{2} & -2 & \phantom{-}\frac{3}{2} \\
\end{bmatrix}.
$$

Note that $\tilde{D}$ is the matrix we would obtain by removing the top row and left column of $D$.  Although the argument above isn't a complete proof, it does provide a good amount of insight into why the eigenvalues we want to consider are those of $-a\tilde{D}$, not those of $-aD$.  

In [ ]:
def plot_eigenvalues(a, nx, cfl, w):

    dx = 0.1
    dt = cfl*(dx/a)

    x = np.arange(nx)*dx

    D = first_deriv_matrix(x, w).toarray()

    D = D[1:,1:]

    scaled_eigs = -a*dt*np.linalg.eigvals(D)

    plt.figure(figsize=(10,5))
    for i in range(2):
        plt.subplot(1, 2, i+1)
        plt.plot(np.real(scaled_eigs), np.imag(scaled_eigs), '.')
        theta = np.linspace(0, 2*np.pi, 1000)
        plt.plot(np.cos(theta)-1, np.sin(theta), '--')
        if i == 0:
            plt.title('Forward Euler')
            plt.axis('equal')
        else:
            plt.xlim([-0.01, 0.01])

    plt.figure(figsize=(10,5))
    for i in range(2):
        plt.subplot(1, 2, i+1)
        plt.plot(np.real(scaled_eigs), np.imag(scaled_eigs), '.')
        xf = np.linspace(-2, 0.1, 200)
        yf = np.linspace(-2, 2, 400)
        Xf, Yf = np.meshgrid(xf, yf)
        Zf = Xf + 1j*Yf
        R = 1 + Zf + Zf**2/2
        mod_R = np.abs(R)
        plt.contour(xf, yf, mod_R, levels=[1], linestyles='dashed')
        if i == 0:
            plt.title('RK2')
            plt.axis('equal')
        else:
            plt.xlim([-0.01, 0.01])

    plt.figure(figsize=(10,5))
    for i in range(2):
        plt.subplot(1, 2, i+1)
        plt.plot(np.real(scaled_eigs), np.imag(scaled_eigs), '.')
        xf = np.linspace(-3, 0.3, 300)
        yf = np.linspace(-3, 3, 600)
        Xf, Yf = np.meshgrid(xf, yf)
        Zf = Xf + 1j*Yf
        R = 1 + Zf + Zf**2/2 + Zf**3/6 + Zf**4/24
        mod_R = np.abs(R)
        plt.contour(xf, yf, mod_R, levels=[1], linestyles='dashed')
        if i == 0:
            plt.title('RK4')
            plt.axis('equal')
        else:
            plt.xlim([-0.01, 0.01])

In [ ]:
plot_eigenvalues(1.8, 50, 2.5, 1)

In [ ]:
plot_eigenvalues(2.2, 150, 1.8, 2)

In [ ]:
plot_eigenvalues(1.8, 50, 1.5, 3)

In [ ]:
plot_eigenvalues(1.8, 50, 1.5, 4)

In [ ]:
plot_eigenvalues(1.8, 50, 1.4, 5)